In [2]:
#Imports

import numpy as np

In [ ]:
#State Variable

x = np.array([0, 1.]) # Initial state (position and velocity)

#Initialising Covariance Matrix to zero because I'm not too sure of how a car's position and velocity would vary
P = np.diag([16., 109.])

dt = 0.1  # Time step (in seconds)

F = np.array([[1, dt], 
              [0, 1]])  # Constant Velocity Model

sigma_acc = 1.0  # Standard deviation of the acceleration (m/s^2);
q = sigma_acc**2
Q = np.array([[dt**4/4, dt**3/2],
                [dt**3/2, dt**2]]) * q

Chose the position of the car to start at the origin and the car to have a velocity of 1ms^-1, because they were simple values.

Then for my Covariance matrix I chose 16.0 because the square root would give me a standard deviation of 4, meaning that I am 99.7% sure of the position of the car ± 12 metres. I believe the maximum speed on the motorway is 70mph so roughly converted that into 31.38 ms-1. This would be the maximum of speed our car could have, so made this equal to 3 times the standard deviation, and then computed the variance to be about 109. Then I chose both the top right and bottom left of the matrix to be 0, because while I know they are related I'm not entirely sure of how I would put this into integer form, so I would let the updating nature of the filter fill this in eventually.

For the Process Model, F, in our constant velocity model the position is given to us by $\bar{x}$ = x + $\dot{x}$, where $\bar{x}$ is our state estimate, and $\dot{x}$ is the derivate of the position, i.e. the velocity. And because velocity is constant we have $\dot{x}$ = $\dot{x}$. So we have $$\bar{x}_k = 1 \cdot x_{k-1} + \Delta t\,\dot{x_{k-1}}$$ and  $$\bar{\dot{x}} = 0 \cdot x_{k - 1} + 1 \cdot \dot{x_{k-1}}$$ this gives us our matrix form. $$\begin{bmatrix} \bar{x} \\ \bar{\dot{x}} \end{bmatrix} = \begin{bmatrix} 1 & \Delta t \\ 0 & 1 \end{bmatrix} \begin{bmatrix} x_{k-1} \\ \dot{x_{k-1}} \end{bmatrix}$$ $$ \bar{x} = Fx $$

<!-- I need to explain how q works>